In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchsummary

In [5]:
class ConvBlock(nn.Module):
    def __init__(
        self, 
        in_channels, 
        out_channels,
        **kwargs,
    ):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(
            in_channels, 
            out_channels,
            **kwargs,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))
    
    
class InceptionBlock(nn.Module):
    def __init__(
        self, 
        in_channel,
        out_1x1,
        red_3x3,
        out_3x3,
        red_5x5,
        out_5x5,
        out_1x1pool,
    ) -> None:
        super(InceptionBlock, self).__init__()
        
        self.branch1 = ConvBlock(in_channel, out_1x1, kernel_size=1)
        self.branch2 = nn.Sequential(
            ConvBlock(in_channel, red_3x3, kernel_size=1),
            ConvBlock(red_3x3, out_3x3, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            ConvBlock(in_channel, red_5x5, kernel_size=1),
            ConvBlock(red_5x5, out_5x5, kernel_size=5, stride=1, padding=2),
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channel, out_1x1pool, kernel_size=1),
        )
        
    def forward(self, x):
        return torch.cat([
            self.branch1(x),
            self.branch2(x),
            self.branch3(x),
            self.branch4(x),
        ], 1)

In [6]:

class GoogleNet(nn.Module):
    def __init__(self, in_channels:int=3, num_classes:int=1000) -> None:
        super().__init__()
        self.conv1 = ConvBlock(
            in_channels=in_channels, 
            out_channels=64, 
            kernel_size=(7,7), 
            stride=(2,2), 
            padding=(3,3),
        )
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = ConvBlock(
            in_channels=64,
            out_channels=192,
            kernel_size=(3,3),
            stride=(1,1),
            padding=(1,1),
        )
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # in_channel -> out_1x1 -> red_3x3 -> out_3x3 -> red_5x5 -> out_5x5 -> out_1x1pool,
        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x
        

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [9]:
model = GoogleNet(in_channels=3, num_classes=1000).to(device)
x = torch.randn(1, 3, 224, 224).to(device)

print(model(x).shape)

torch.Size([1, 1000])


In [12]:
torchsummary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         ConvBlock-4         [-1, 64, 112, 112]               0
         MaxPool2d-5           [-1, 64, 56, 56]               0
            Conv2d-6          [-1, 192, 56, 56]         110,784
       BatchNorm2d-7          [-1, 192, 56, 56]             384
              ReLU-8          [-1, 192, 56, 56]               0
         ConvBlock-9          [-1, 192, 56, 56]               0
        MaxPool2d-10          [-1, 192, 28, 28]               0
           Conv2d-11           [-1, 64, 28, 28]          12,352
      BatchNorm2d-12           [-1, 64, 28, 28]             128
             ReLU-13           [-1, 64, 28, 28]               0
        ConvBlock-14           [-1, 64,